In [1]:
from format_data import format_2D
from models import Autoencoder, train_model, Transformer, SmallAutoencoder
from torch.optim import Adam
from torch.nn import MSELoss
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import numpy as np 
from data_analysis import plot_property_distribution, plot_property_distribution2
# from torchsummary import summary
from torchinfo import summary
import torch
import constants as c
import pandas as pd 
from model_analysis import plot_loss, plot_anomaly_score, plot_roc

In [2]:
background_label = "QCD"
signal_label = "WJet"
props = ['dzErr', 'pt']
data_dir = "/isilon/data/users/jpfeife2/AutoEncoder-Anomaly-Detection/processed_data"

batch_size = 20
epochs = 10
initial_lr = 0.001
weight_decay = 1e-3
latent_dim = 12

n_props = len(props)
# n_props = 8
# props = ['-211', '-13', '-11', '11', '13', '22', '130', '211']

prop_string = ''.join(['_' + str(prop) for prop in props])

In [3]:
background_file = data_dir + "/background.pkl"
signal_file = data_dir + "/signal.pkl"

In [5]:
background_ = pd.read_pickle(background_file)

: 

: 

: 

In [4]:
signal_ = pd.read_pickle(signal_file)

In [ ]:
from format_data import one_hot_encode_list, make_histogram
import pandas as pd 
from data_analysis import make_histogram
import numpy as np 
# import dgl #deep graph network that will make integrating graphs easier
import torch
from scipy.spatial.distance import cdist
import constants as c
import torch.nn as nn 
import torch.nn.functional as F
from sklearn import preprocessing

In [ ]:
def format_2D(data, properties, scalers=None):
    hists = []
    n_properties = len(properties)
    if scalers == None: 
        scalers = []
    vals = []

    for i, prop in enumerate(properties): 
        vals.append([])

        if prop == 'pdgId': 
            flattened_list = [item for sublist in data['pdgId'] for item in sublist]
            unique_values_list = sorted(list(set(flattened_list)))
            # value_to_index = {value: index + 1 for index, value in enumerate(unique_values_list)}
            one_hot_dict = one_hot_encode_list(unique_values_list, data['pdgId'])
            particle_ids = []
            for key in one_hot_dict: 
                one_hot_list = one_hot_dict[key]
                # hists.append([make_histogram(data['eta'][i], data['phi'][i],
                #                     [value_to_index[number] for number in one_hot_list[i]]) for i in range(data.shape[0])])
                hists.append([make_histogram(data['eta'][j], data['phi'][j], one_hot_list[j]) for j in range(data.shape[0])])
                particle_ids.append(key)
            n_properties += len(hists) - 1
        elif prop[-3:] == "Err": 
            '''
            currently only makes sense for dz and d0 because they have 
            '''
            prop1 = prop[:2]
            valid_pdg = [-11, 11, -13, 13, -211, 211]
            all_vals = [p / z for sublist1, sublist2, sublist3 in zip(data[prop1], data[prop], data['pdgId']) for p, z, x in zip(sublist1, sublist2, sublist3) if z >=0 and x in valid_pdg and np.abs(p/z) <= 5.0]
            
            if scalers == []:
                scaler = preprocessing.StandardScaler().fit(np.array(all_vals).reshape(-1,1))
                scalers.append(scaler)

            hist_list = []
            for j in range(data.shape[0]): 
                prop_data = pd.to_numeric(data[prop1][j])/pd.to_numeric(data[prop][j])
                valid_indices = np.array(data[prop][j]) >= 0 
                
                # checks that the error is greater than 0 and that it is within 5sigma of the original distribution
                prop_data = np.array(prop_data)[valid_indices]
                within5sigmaindices = np.where(np.abs(prop_data) <= 5.0)
                prop_data = prop_data[within5sigmaindices]
                if len(prop_data) > 0: 
                
                    prop_scaled = scalers[i].transform(prop_data.reshape(-1,1))
                    filtered_eta = np.array(data['eta'][j])[valid_indices][within5sigmaindices].flatten()
                    filtered_phi = np.array(data['phi'][j])[valid_indices][within5sigmaindices].flatten()
                    filtered_prop = np.array(prop_scaled).flatten()
                    vals[i].extend(filtered_prop)
                    hist_list.append(make_histogram(filtered_eta, filtered_phi, filtered_prop))
                
                else: 
                    print('error, length of data is ', len(prop_data))
                    hist_list.append(make_histogram([0], [0], [0]))
            hists.append(hist_list)

        else: 
            flattened_list = [item*500 for sublist in data[prop] for item in sublist]
            unique_values_list = sorted(list(set(flattened_list)))
            if len(scalers) < n_properties: 
                scaler = preprocessing.StandardScaler(with_mean=False).fit(np.array(flattened_list).reshape(-1,1))
                scalers.append(scaler)
            if 0.0 in unique_values_list: 
                print('warning, 0 in data will cause incorrect data translation to histograms')
            hists.append([make_histogram(data['eta'][j], data['phi'][j], scalers[i].transform(np.array(data[prop][j]).reshape(-1,1)).flatten()*300) for j in range(data.shape[0])])
            vals[i].extend(flattened_list)
            

            # print("warning: multiplying by 10")
    total_hist = np.stack((hists), axis=-1)
    total_hist = np.reshape(total_hist, (-1, c.BINS, c.BINS, n_properties)).astype('float32')

    print("Length of data: ", len(total_hist))
    return total_hist, scalers, vals

In [ ]:
background, scalers, background_data = format_2D(background_, properties=props, scalers=None)
np.save(data_dir + "/background" + prop_string + ".npy", background)
print("BACKGROUND LOADED AND SAVED")

NameError: name 'background_' is not defined

In [ ]:
signal, _, signal_data = format_2D(signal_, properties=props, scalers=scalers)
np.save(data_dir + "/signal" + prop_string + ".npy", signal)
print("SIGNAL LOADED AND SAVED")

error, length of data is  0
error, length of data is  0
error, length of data is  0
error, length of data is  0
error, length of data is  0
Length of data:  17896
SIGNAL LOADED AND SAVED


In [ ]:
for i, prop in enumerate(props): 
    plot_property_distribution2(background_data[i], signal_data[i], prop, background_label, signal_label)

In [ ]:
print(np.mean([i * i for i in background_data[0]]))
print(np.mean([i * i for i in background_data[1]]))

1.0000006334663487
757.3946116746873


In [ ]:
print(np.mean([i * i for i in signal_data[0]]))
print(np.mean([i * i for i in signal_data[1]]))

1.1015872597764698
852.764850994828


In [ ]:
pt = [i for i in signal_data[1]]

In [ ]:
m = np.mean(pt)
std = np.std(pt)

In [ ]:
pt = [i for i in pt if (i - m) <= 3 * std]

In [ ]:
print(np.mean([i * i for i in signal_data[0]]))
print(np.mean([i * i * 30*30/ (500 * 500) for i in pt]))

1.1015872597764698
1.0561193452958901
